In [21]:
import spacy
import glob
import pandas as pd

nlp = spacy.load('en_core_web_sm')


path = './Biographies/*'
files = sum([glob.glob(dir+'/*.txt') for dir in glob.glob(path)],[])

filenames = [file.split('/')[3] for file in files]

ents=[]
for file in files:
    with open(file) as infile:
        file_content =  infile.read()
        doc = nlp(file_content)
        ents.append(doc.ents)

data = list(zip(filenames,ents))
df = pd.DataFrame(data, columns = ['filename','ents'])
print(df.head())

                                    filename  \
0        AquileoJ.Echeverría_Journalists.txt   
1  ThomasHenryShadwellClerke_Journalists.txt   
2                 Journalist_Journalists.txt   
3             KayodeAkintemi_Journalists.txt   
4                  31Minutos_Journalists.txt   

                                                ents  
0  ((Aquileo, J., Echeverría), (May, 22, ,, 1866)...  
1  ((Thomas, Henry, Shadwell, Clerke), (KH), (Iri...  
2  ((Matthew, C., Nisbet), (Walter, Lippmann), (F...  
3  ((Kayode, Akintemi), (June, 26, ,, 1965), (Nig...  
4  ((31), (English), (31, minutes), (Chilean), (A...  


In [22]:
import stanza
stanza.download('en')
nlp = stanza.Pipeline(lang="en") 



/home/marina/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-11 15:22:51 INFO: Downloaded file to /home/marina/stanza_resources/resources.json
2024-06-11 15:22:51 INFO: Downloading default packages for language: en (English) ...
2024-06-11 15:23:11 INFO: Downloaded file to /home/marina/stanza_resources/en/default.zip
2024-06-11 15:23:16 INFO: Finished downloading models and saved to /home/marina/stanza_resources
2024-06-11 15:23:16 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-06-11 15:23:17 INFO: Downloaded file to /home/marina/stanza_resources/resources.json
2024-06-11 15:23:18 INFO: Loading these models for language: en (English):
| Pr